In [2]:
import pandas as pd

In [3]:
print("hello world")

hello world


In [4]:
# 엑셀파일 읽어오기
xl=pd.read_excel('tobacco_data.xlsx')

In [34]:
print(xl.columns)

Index(['idx', 'grpid', 'resid', 'fwid', 'mobileaccess', 'browser', 'UserAgent',
       'st_date', 'reg_date', 'total_time', 'rtpage', 'onpage', 'OutMarker',
       'SQ1', 'SQ2', 'SQ2_1', 'SQ3', 'SQ3_1', 'SQ4', 'SQ5', 'AQ0', 'BQ0',
       'AQ1', 'AQ2', 'AQ3_1', 'AQ4', 'AQ5', 'AQ6_1', 'AQ7', 'AQ8', 'AQ9',
       'AQ10', 'AQ11', 'AQ12', 'AQ13', 'AQ14', 'AQ15', 'AQ16', 'AQ17',
       'AQ17_1', 'AQ17_2', 'AQ18', 'BQ1', 'BQ2', 'BQ3_1', 'BQ4', 'BQ5',
       'BQ6_1', 'BQ7', 'BQ8', 'BQ9', 'BQ10', 'BQ11', 'BQ12', 'BQ13', 'BQ14',
       'BQ15', 'BQ16', 'BQ17', 'BQ17_1', 'BQ17_2', 'BQ18'],
      dtype='object')


In [69]:
# 필요없는 변수 리스트
dellist = ['idx','grpid','resid','fwid','mobileaccess','browser','UserAgent','st_date','reg_date','total_time','rtpage','onpage','OutMarker','SQ1','SQ2','SQ2_1','SQ3','SQ3_1','SQ4','SQ5','AQ0','BQ0','AQ17_1','AQ17_2','BQ17_1','BQ17_2']

# 필요없는 변수 제거
xl2=xl.drop(dellist,axis=1)
    
# jrs 계산 기본 점수에 -4 (Q3_1,Q6_1)
jrslist = ['AQ3_1','AQ6_1','BQ3_1','BQ6_1']
for i in jrslist:
    xl2[i]=xl2[i].sub(4)

In [70]:
# top1% 부터 bottom3%, 평균값까지 통계값 구하는 계산식
def calStat(df,ord,num):
    #num = 7 또는 3(jrs), 7점척도 기준임
    totcnt = len(df)
    top1cnt = len(df[df[ord]==num])/totcnt*100
    top2cnt = len(df[(df[ord]==num) | (df[ord]==num-1)])/totcnt*100
    top3cnt = len(df[(df[ord]==num) | (df[ord]==num-1) | (df[ord]==num-2)])/totcnt*100
    bottom1cnt = len(df[df[ord]==num-6])/totcnt*100
    bottom2cnt = len(df[(df[ord]==num-6) | (df[ord]==num-5)])/totcnt*100
    bottom3cnt = len(df[(df[ord]==num-6) | (df[ord]==num-5) | (df[ord]==num-4)])/totcnt*100
    totmean = df[ord].mean()
    
    if num==3:
        #jrs문항의 경우 따로 (-1,0,1) 통계값 구해야함
        jr3 = len(df[(df[ord]==1) | (df[ord]==0) | (df[ord]==-1)])/totcnt*100
        jr0 = len(df[df[ord]==0])/totcnt*100
        result = [top1cnt,top2cnt,top3cnt,jr3,jr0,bottom1cnt,bottom2cnt,bottom3cnt,totmean]
    else:
        result = [top1cnt,top2cnt,top3cnt,bottom1cnt,bottom2cnt,bottom3cnt,totmean]
        
    return result


# 새로운 데이터프레임 만들어줄 빈 딕셔너리 만들어놓기
xl3={}
xl3_jrs={}
jrs_columns=[]
xl3_columns=[]
# 각 변수별 통계값 구하여 빈 딕셔너리에 넣어주기
for i in xl2.columns:
    if len(i.split('_'))>1: #jrs문항이면 따로 빼주기
        xl3_jrs[i] = xl2.pop(i) # Series 형태로 빠지게 된다 추후 데이터 프레임형식으로 변환해주어야 함
        jrs_columns.append(i)
    else:
        xl3[i]=calStat(xl2,i,7)
        xl3_columns.append(i)

# 통계값이 들어간 딕셔너리를 데이터프레임으로 변환(행이름(index),열이름(columns) 직접 붙여주기)
xl_tot=pd.DataFrame(xl3,index=['top1cnt','top2cnt','top3cnt','bottom1cnt','bottom2cnt','bottom3cnt','Mean'],columns=xl3_columns)
print(xl_tot)

#jrs문항
xl4_jrs=pd.DataFrame(xl3_jrs,columns=jrs_columns) # 앞에서 Series형태로 빼주었기때문에 데이터 프레임으로 변환
xl5_jrs={}

for i in jrs_columns:
    xl5_jrs[i]=calStat(xl4_jrs,i,3)

xl_tot_jrs=pd.DataFrame(xl5_jrs,index=['TM%(+3)','TM%(+2,+3)','TM%(+1,+2,+3)','JR%(-1,0,+1)','JR%(0)','TL%(-3)','TL%(-2,-3)','TL%(-1,-2,-3)','Mean'],columns=jrs_columns)
print(xl_tot_jrs)


# xl_tot는 일반문항 통계 df
# xl_tot_jrs는 jrs문항 통계 df


                  AQ1        AQ2        AQ4        AQ5        AQ7        AQ8  \
top1cnt     13.333333  13.333333   6.666667  10.000000   0.000000   0.000000   
top2cnt     50.000000  33.333333  20.000000  36.666667  36.666667  53.333333   
top3cnt     86.666667  73.333333  50.000000  73.333333  70.000000  83.333333   
bottom1cnt   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
bottom2cnt   3.333333   3.333333   0.000000   3.333333   3.333333   0.000000   
bottom3cnt   3.333333   6.666667  10.000000  10.000000  10.000000   6.666667   
Mean         5.433333   5.100000   4.666667   5.066667   4.933333   5.300000   

                  AQ9       AQ10       AQ11       AQ12  ...        BQ9  \
top1cnt     23.333333  13.333333   6.666667   6.666667  ...  16.666667   
top2cnt     60.000000  53.333333  46.666667  36.666667  ...  53.333333   
top3cnt     90.000000  83.333333  76.666667  56.666667  ...  80.000000   
bottom1cnt   0.000000   0.000000   0.000000   0.000000  ...   0

In [71]:
#엑셀 파일에 여러 개 시트로 생성
writer=pd.ExcelWriter('report_tobacco.xlsx', engine='openpyxl')

xl_tot.to_excel(writer, sheet_name=('전체 '+str(totcnt))+'명')
xl_tot_jrs.to_excel(writer, sheet_name=('JRS 전체 '+str(totcnt))+'명')

writer.save()

In [ ]:
#엑셀 파일에 여러 개 시트로 생성
 
 
#1. 파일 생성
# writer=pd.ExcelWriter('테스트파일명.xlsx', engine='openpyxl')
 
#2. 생성 파일에 시트명 지정 후 dataframe에 저장한 결과값 넣기
# df1.to_excel(writer, sheet_name='sheet1')
# df2.to_excel(writer, sheet_name='sheet2')
# df3.to_excel(writer, sheet_name='sheet3')
# df4.to_excel(writer, sheet_name='sheet4')
 
#3. 작성 완료 후 파일 저장
# writer.save()
